<a href="https://colab.research.google.com/github/muhammadwajeeharif/Predicting-house-prices-a-regression-example/blob/main/Predicting_house_prices_a_regression_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [3]:
 train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

 **Preparing the data**

In [6]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

**Building your network**

In [7]:
from tensorflow.keras import models
from tensorflow.keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

**K-fold validation**

In [8]:
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)

  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i + 1) * num_val_samples:]],
  axis=0)

  model = build_model()

  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=1)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
  all_scores.append(val_mae)

processing fold # 0
Epoch 1/100
303/303 [==============================] - 1s 1ms/step - loss: 363.7583 - mae: 15.8298
Epoch 2/100
303/303 [==============================] - 0s 1ms/step - loss: 30.7904 - mae: 3.7564
Epoch 3/100
303/303 [==============================] - 0s 1ms/step - loss: 20.5095 - mae: 3.1843
Epoch 4/100
303/303 [==============================] - 0s 985us/step - loss: 14.6126 - mae: 2.6419
Epoch 5/100
303/303 [==============================] - 0s 1ms/step - loss: 15.9714 - mae: 2.5736
Epoch 6/100
303/303 [==============================] - 0s 995us/step - loss: 16.3901 - mae: 2.5181
Epoch 7/100
303/303 [==============================] - 0s 1ms/step - loss: 12.2606 - mae: 2.3141
Epoch 8/100
303/303 [==============================] - 0s 1ms/step - loss: 9.4006 - mae: 2.2873
Epoch 9/100
303/303 [==============================] - 0s 1ms/step - loss: 15.5117 - mae: 2.3998
Epoch 10/100
303/303 [==============================] - 0s 1ms/step - loss: 9.9147 - mae: 2.2785
Epoch

In [9]:
all_scores


[2.0877208709716797, 2.4617161750793457, 2.550705909729004, 2.34525203704834]

In [10]:
np.mean(all_scores)

2.3613487482070923

In [11]:

model = build_model()
model.fit(train_data, train_targets,
epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

4/4 [==============================] - 0s 4ms/step - loss: 18.0871 - mae: 2.5844


In [12]:
test_mae_score


2.5843608379364014